# 📝 Smart Lorem Ipsum Generator

## Generate Meaningful Placeholder Text That Actually Makes Sense!

Lorem Ipsum is gibberish Latin text. This tool generates **coherent, readable placeholder text** which is BETTER!😁😁😁.


In [ ]:
import os
import random
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

load_dotenv(override=True)

openai_key = os.getenv('OPENAI_API_KEY')
if openai_key:
    print(f"OpenAI API Key exists and begins {openai_key[:8]}")
else:
    print("OpenAI API Key not set")


In [ ]:

AVAILABLE_MODELS = {
    "Ollama (Free)": "llama3.2"
}

clients = {}

clients["ollama"] = OpenAI(
    api_key="ollama",
    base_url="http://localhost:11434/v1"
)

# Check if OpenAI key is valid and add OpenAI models
if openai_key:
    clients["openai"] = OpenAI(api_key=openai_key)

    # OpenAI models
    AVAILABLE_MODELS.update({
        "GPT-4o Mini": "gpt-4o-mini",
        "GPT-4o": "gpt-4o",
        "GPT-4 Turbo": "gpt-4-turbo-preview"
    })

In [ ]:
# Categories & Prompts

CONTENT_CATEGORIES = {
    "📰 Blog Post": {
        "description": "Casual, engaging blog content",
        "topics": [
            "productivity tips for remote workers",
            "healthy eating on a budget",
            "traveling solo for the first time",
            "sustainable living practices",
            "building better daily habits",
            "learning new skills as an adult",
            "finding work-life balance",
            "minimalism and decluttering"
        ],
        "style": "conversational, personal, with practical tips"
    },
    "📱 Product Description": {
        "description": "Marketing copy for products",
        "topics": [
            "wireless noise-canceling headphones",
            "organic skincare serum",
            "ergonomic office chair",
            "smart home security camera",
            "sustainable bamboo water bottle",
            "premium leather laptop bag",
            "high-performance running shoes",
            "artisan coffee subscription"
        ],
        "style": "persuasive, benefit-focused, with features and specifications"
    },
    "📊 Technical Documentation": {
        "description": "Clear technical writing",
        "topics": [
            "API authentication methods",
            "database indexing strategies",
            "responsive web design principles",
            "containerization with Docker",
            "RESTful API best practices",
            "version control workflows",
            "security testing procedures",
            "cloud deployment strategies"
        ],
        "style": "precise, instructional, with examples and technical terms"
    },
    "📰 News Article": {
        "description": "Journalistic news content",
        "topics": [
            "renewable energy adoption trends",
            "artificial intelligence in healthcare",
            "urban farming initiatives",
            "electric vehicle market growth",
            "remote work policy changes",
            "sustainable fashion movements",
            "space exploration advances",
            "educational technology innovations"
        ],
        "style": "objective, factual, inverted pyramid structure"
    },
    "✉️ Business Email": {
        "description": "Professional email communication",
        "topics": [
            "project status update to stakeholders",
            "meeting invitation for quarterly review",
            "client proposal follow-up",
            "team collaboration request",
            "vendor contract negotiation",
            "customer service response",
            "partnership opportunity inquiry",
            "employee recognition announcement"
        ],
        "style": "professional, concise, action-oriented"
    },
    "📖 Short Story": {
        "description": "Creative narrative fiction",
        "topics": [
            "a mysterious package arrives",
            "unlikely friendship forms",
            "last day before big change",
            "discovering hidden family secret",
            "chance encounter changes everything",
            "childhood memory resurfaces",
            "ordinary day becomes extraordinary",
            "crossroads decision moment"
        ],
        "style": "narrative, descriptive, with dialogue and character development"
    },
    "🎯 Marketing Copy": {
        "description": "Persuasive advertising content",
        "topics": [
            "limited-time software discount",
            "new fitness program launch",
            "premium membership benefits",
            "eco-friendly brand campaign",
            "exclusive event invitation",
            "product upgrade announcement",
            "customer loyalty program",
            "seasonal sale promotion"
        ],
        "style": "compelling, benefit-driven, with clear call-to-action"
    },
    "📝 How-To Guide": {
        "description": "Step-by-step instructions",
        "topics": [
            "setting up home office",
            "meal prep for beginners",
            "basic photography techniques",
            "starting meditation practice",
            "organizing digital files",
            "creating budget spreadsheet",
            "building raised garden bed",
            "troubleshooting common issues"
        ],
        "style": "instructional, clear, numbered steps with helpful tips"
    },
    "🎓 Educational Content": {
        "description": "Teaching and learning material",
        "topics": [
            "introduction to machine learning",
            "understanding compound interest",
            "basics of color theory",
            "fundamentals of nutrition",
            "critical thinking skills",
            "effective communication strategies",
            "time management principles",
            "emotional intelligence development"
        ],
        "style": "explanatory, engaging, with examples and key takeaways"
    },
    "💼 Case Study": {
        "description": "Business success stories",
        "topics": [
            "startup scaling challenges",
            "digital transformation project",
            "customer retention strategy",
            "process optimization initiative",
            "brand repositioning campaign",
            "crisis management response",
            "innovation implementation",
            "team productivity improvement"
        ],
        "style": "analytical, results-focused, problem-solution-outcome structure"
    },
    "🌟 Social Media Post": {
        "description": "Engaging social content",
        "topics": [
            "behind-the-scenes company culture",
            "customer success celebration",
            "industry trend commentary",
            "helpful quick tip",
            "thought-provoking question",
            "product feature highlight",
            "community engagement poll",
            "inspirational quote reflection"
        ],
        "style": "brief, conversational, engaging with emojis and hashtags"
    },
    "📄 White Paper": {
        "description": "In-depth analytical report",
        "topics": [
            "cybersecurity threat landscape",
            "customer behavior analytics",
            "supply chain optimization",
            "emerging market opportunities",
            "regulatory compliance frameworks",
            "sustainability metrics",
            "technology adoption patterns",
            "workforce development trends"
        ],
        "style": "authoritative, data-driven, formal with research citations"
    }
}

print(f"Loaded {len(CONTENT_CATEGORIES)} content categories")
print(f"   Total topic variations: {sum(len(cat['topics']) for cat in CONTENT_CATEGORIES.values())}")

In [ ]:

# Text Generation

def generate_smart_lorem(
    category: str,
    num_paragraphs: int,
    model_choice: str,
    creativity: float,
    custom_topic: str = ""
):
    """
    Generate placeholder text
    
    Args:
        category: Content category (e.g., "Blog Post", "Product Description")
        num_paragraphs: Number of paragraphs to generate (1-10)
        model_choice: AI model to use
        creativity: Temperature for generation (0.3-1.5)
        custom_topic: Optional custom topic override
    
    Returns:
        Generated text string
    """
    
    # Get category config
    cat_config = CONTENT_CATEGORIES[category]
    
    # Select topic (custom or random from category)
    if custom_topic and custom_topic.strip():
        topic = custom_topic.strip()
    else:
        topic = random.choice(cat_config["topics"])
    
    # Build prompt
    system_prompt = f"""
You are a professional content writer creating placeholder text for design and development.

Generate realistic, coherent text that reads naturally and makes sense.
DO NOT use Lorem Ipsum or gibberish.
DO NOT add titles, headers, or markdown formatting unless it's part of the natural content.
DO NOT explain what you're doing - just generate the content.

Style: {cat_config['style']}
Category: {cat_config['description']}
"""
    
    user_prompt = f"""
Write {num_paragraphs} paragraph{'s' if num_paragraphs > 1 else ''} about: {topic}

Requirements:
- Make it realistic and coherent
- Use natural language that makes sense
- Match the style: {cat_config['style']}
- Each paragraph should be 3-5 sentences
- Separate paragraphs with double line breaks
- DO NOT add any titles, headers, or formatting
"""
    
    # Get model ID
    model_id = AVAILABLE_MODELS[model_choice]
    
    # Determine which client to use based on model
    if "gpt-" in model_id or model_id.startswith("gpt"):
        # OpenAI model
        client = clients.get("openai")
        if not client:
            return """❌ Error: OpenAI model selected but no API key configured.

Please add your OPENAI_API_KEY in the configuration cell, or select a free Ollama model."""
    else:
        # Ollama
        client = clients.get("ollama")

    try:
        # Generate text
        response = client.chat.completions.create(
            model=model_id,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=creativity,
            max_tokens=2000
        )

        generated_text = response.choices[0].message.content.strip()

        return generated_text

    except Exception as e:
        return f"Error generating text: {str(e)}"

print("Text generation ready")

In [ ]:
# Custom CSS for better styling
custom_css = """
.output-box {
    font-family: 'Georgia', serif;
    line-height: 1.8;
    font-size: 16px;
}
.category-info {
    background: #f0f0f0;
    padding: 10px;
    border-radius: 5px;
    margin: 10px 0;
}
"""

# Build Gradio interface
with gr.Blocks(theme=gr.themes.Soft(), css=custom_css, title="Smart Lorem Ipsum") as demo:
    
    gr.Markdown("""
    # Smart Lorem Ipsum Generator
    
    Generate **meaningful placeholder text** instead of gibberish Latin. Perfect for mockups, prototypes, and testing!
    
    Choose a category, select your AI model, and get coherent, realistic text instantly.
    
    🎉 **Free Ollama model always available!** Add OpenAI key to unlock GPT models.
    """)
    
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### ⚙️ Configuration")
            
            category_dropdown = gr.Dropdown(
                choices=list(CONTENT_CATEGORIES.keys()),
                value=list(CONTENT_CATEGORIES.keys())[0],
                label="📂 Content Category",
                info="Type of content to generate"
            )
            
            # Display category description
            category_info = gr.Markdown(
                value=f"**{CONTENT_CATEGORIES[list(CONTENT_CATEGORIES.keys())[0]]['description']}**",
                elem_classes="category-info"
            )
            
            num_paragraphs = gr.Slider(
                minimum=1,
                maximum=10,
                value=1,
                step=1,
                label="📏 Number of Paragraphs",
            )
            
            model_dropdown = gr.Dropdown(
                choices=list(AVAILABLE_MODELS.keys()),
                value=list(AVAILABLE_MODELS.keys())[0],
                label="🤖 AI Model",
            )
            
            creativity_slider = gr.Slider(
                minimum=0.3,
                maximum=1.5,
                value=0.8,
                step=0.1,
                label="🎨 Creativity Level",
                info="Lower = predictable, Higher = creative"
            )
            
            custom_topic = gr.Textbox(
                label="💡 Custom Topic (Optional)",
                placeholder="e.g., 'benefits of morning routines'...",
                lines=2,
                info="Leave blank for random topic"
            )
            
            generate_btn = gr.Button(
                "✨ Generate Smart Lorem Ipsum",
                variant="primary",
                size="lg"
            )
        
        with gr.Column(scale=2):
            gr.Markdown("### 📄 Generated Text")
            
            output_text = gr.Textbox(
                label="Your Placeholder Text",
                lines=20,
                max_lines=30,
                show_copy_button=True,
                elem_classes="output-box",
                placeholder="Your generated text will appear here...\n\nClick the button to generate!"
            )
            
            with gr.Row():
                word_count = gr.Textbox(
                    label="📊 Word Count",
                    value="0",
                    interactive=False,
                    scale=1
                )
                char_count = gr.Textbox(
                    label="📊 Character Count",
                    value="0",
                    interactive=False,
                    scale=1
                )
    
    # Examples section
    gr.Markdown("""
    ---
    ### 💡 Quick Start Examples
    Click any example below to try it:
    """)
    
    gr.Examples(
        examples=[
            ["📰 Blog Post", 2, list(AVAILABLE_MODELS.keys())[0], 0.8, ""],
            ["📱 Product Description", 1, list(AVAILABLE_MODELS.keys())[0], 0.7, "wireless earbuds"],
            ["📊 Technical Documentation", 3, list(AVAILABLE_MODELS.keys())[0], 0.5, ""],
            ["📖 Short Story", 4, list(AVAILABLE_MODELS.keys())[0], 1.2, "a rainy evening"],
            ["🎯 Marketing Copy", 2, list(AVAILABLE_MODELS.keys())[0], 0.9, ""],
        ],
        inputs=[category_dropdown, num_paragraphs, model_dropdown, creativity_slider, custom_topic],
        label="Try these:"
    )
    
    # Update category description when category changes
    def update_category_info(category):
        return f"**{CONTENT_CATEGORIES[category]['description']}**"
    
    category_dropdown.change(
        fn=update_category_info,
        inputs=[category_dropdown],
        outputs=[category_info]
    )
    
    # Generate button action
    def generate_and_count(category, num_para, model, creativity, custom):
        text = generate_smart_lorem(category, num_para, model, creativity, custom)
        # Calculate stats (excluding metadata)
        main_text = text.split('\n\n---\n')[0] if '\n\n---\n' in text else text
        words = len(main_text.split())
        chars = len(main_text)
        return text, str(words), str(chars)
    
    generate_btn.click(
        fn=generate_and_count,
        inputs=[category_dropdown, num_paragraphs, model_dropdown, creativity_slider, custom_topic],
        outputs=[output_text, word_count, char_count]
    )

# Launch the app
if __name__ == "__main__":
    demo.launch(
        share=False,
        inbrowser=True,
        show_error=True
    )